## Training a neural network to classify gender on NoRec dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import nltk
import tensorflow_datasets as tfds

## Loading the dataset

In [2]:
train_dir = 'corpus/data/train'
dev_dir = 'corpus/data/dev'
test_dir = 'corpus/data/test'

batch_size = 32
buffer_size = 10000
seed = 42

In [3]:
raw_train_ds = keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    seed=42)

Found 3397 files belonging to 2 classes.


In [4]:
raw_dev_ds = keras.preprocessing.text_dataset_from_directory(
    dev_dir,
    batch_size=batch_size,
    seed=42)

raw_test_ds = keras.preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size,
    seed=42)

Found 428 files belonging to 2 classes.
Found 428 files belonging to 2 classes.


In [5]:
for example, label in raw_train_ds.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b'Bokanmeldelse:Lars Joachim Grimstad: \n\xc2\xabStatsminister Fahr & s\xc3\xb8nn. \nEgoland\xc2\xbb \n\nEn barnebok med smart og underholdende politisk satire for nordmenn i alle aldre. \n\nN\xc3\xa5 er den tredje boken om statsminister Fahr & s\xc3\xb8nn her, og den er et herlig harselas med norsk selvgodhet og norsk politikk. \n\nTeddy Fahr kj\xc3\xb8rte taxi f\xc3\xb8r han dannet Merpartiet og ble valgt til statsminister. \nHans filosofi er at nordmenn fortjener \xc3\xa5 kose seg og at penger er til for \xc3\xa5 brukes. \nHan innf\xc3\xb8rer Teddy-timen \xe2\x80\x93 en time ekstra i senga til alle, bygger vannsklie p\xc3\xa5 operabygget og omgj\xc3\xb8r Bislett stadion til en virtuell vikingpark. \nHans higen etter popularitet gj\xc3\xb8r ham latterlig, men ogs\xc3\xa5 dypt menneskelig. \nDen virkelige helten er s\xc3\xb8nnen hans, Finn. \n\nFinn er f\xc3\xb8lsom, intelligent, snill, beskjeden og pinglete \xe2\x80\x93 egentlig en klassisk barnehelt. \nSom i mange barneb\xc

In [6]:
VOCAB_SIZE=1000
encoder = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)

encoder.adapt(raw_train_ds.map(lambda text, label: text))

In [16]:
vocab = np.array(encoder.get_vocabulary())
vocab

array(['', '[UNK]', 'og', 'i', 'er', 'som', 'en', 'det', 'av', 'på',
       'til', 'å', 'med', 'for', 'den', 'om', 'har', 'et', 'at', 'han',
       'de', 'ikke', 'men', 'seg', 'fra', 'hun', 'også', 'blir', 'så',
       'kan', 'vi', 'denne', 'var', 'sin', 'dette', 'etter', 'jeg', 'der',
       'eller', 'når', 'selv', 'mange', 'ut', 'mer', 'noe', 'boka', 'får',
       'over', 'boken', 'her', 'opp', 'hans', 'andre', 'skal', '–', 'bok',
       'alle', 'være', 'vil', 'år', 'inn', 'man', 'noen', 'både', 'ble',
       'enn', 'bare', 'da', 'mot', 'mellom', 'ved', 'to', 'mye', 'roman',
       'alt', 'forfatteren', 'helt', 'godt', 'gjør', 'ha', 'gjennom',
       'litt', 'hva', 'romanen', 'nå', 'sine', 'må', 'går', 'du', 'liv',
       'sitt', 'slik', 'dem', 'sider', 'bli', 'skriver', 'første',
       'hadde', 'kommer', 'flere', 'hvordan', 'nok', 'få', 'store',
       'hele', 'uten', 'hvor', 'kunne', 'under', 'før', 'oss', 'ham',
       'livet', 'vært', 'sammen', 'nye', 'hennes', 'kanskje', 'mest'

In [25]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(raw_train_ds, epochs=20,
                    validation_data=raw_dev_ds, 
                    validation_steps=10)

Epoch 1/20
107/107 [==============================] - 83s 777ms/step - loss: 0.6751 - accuracy: 0.4051 - val_loss: 0.7388 - val_accuracy: 0.5344
Epoch 2/20
107/107 [==============================] - 85s 791ms/step - loss: 0.6746 - accuracy: 0.4024 - val_loss: 0.7286 - val_accuracy: 0.5188
Epoch 3/20
107/107 [==============================] - 87s 813ms/step - loss: 0.6738 - accuracy: 0.4024 - val_loss: 0.7217 - val_accuracy: 0.5188
Epoch 4/20
107/107 [==============================] - 84s 785ms/step - loss: 0.6721 - accuracy: 0.4024 - val_loss: 0.7141 - val_accuracy: 0.5063
Epoch 5/20
107/107 [==============================] - 85s 793ms/step - loss: 0.6674 - accuracy: 0.4345 - val_loss: 0.7030 - val_accuracy: 0.6344
Epoch 6/20
107/107 [==============================] - 84s 786ms/step - loss: 0.6462 - accuracy: 0.6618 - val_loss: 0.7324 - val_accuracy: 0.6750
Epoch 7/20
107/107 [==============================] - 80s 745ms/step - loss: 0.6525 - accuracy: 0.6217 - val_loss: 0.6945 - val_ac

In [26]:
model.predict(np.array(["Sandkassespill er noe for seg selv, og det er en god grunn til at de er blitt så populære. De tillater utforskning og forlengede spilletider som smalere, mer lineære opplevelser bare ikke kan, og i univers hvor innlevelse er viktig er det ikke så rart at utviklere velger å tilby en sandkasse der spilleren får utfolde seg og bruke så mye tid som mulig. Sandkassespill kommer imidlertid gjerne med en hake; hovedfortellingen, som gjerne fungerer som en overordnet historie i motsetning til mindre sidefortellinger, får gjerne gjennomgå. Det jeg mener med dette er at sandkassespill gjerne oppfordrer til å gjøre andre ting enn det hovedfiguren og plottet gjerne insinuerer at man burde gjøre, nettopp fordi det er så mye å finne på og oppsøke utenom denne fortellingen. Kanskje må man samle mer erfaringspoeng for å kunne hamle opp med en ekstra sterk skurk, kanskje kreves det mer penger for å låse opp bedre våpen, og muligens er det enda kulere, korte sidefortellinger som sjarmerer mer enn hovedfortellingen gjør på samme tidspunkt."]))

array([[0.00308603]], dtype=float32)

In [29]:
model.save("lstm")

INFO:tensorflow:Assets written to: lstm/assets


INFO:tensorflow:Assets written to: lstm/assets


In [57]:
zeros = 0
ones = 0
for _, i in raw_test_ds.as_numpy_iterator():
    zeros += (i == 0).sum()
    ones += (i == 1).sum()
zeros, ones

(196, 232)